<a href="https://colab.research.google.com/drive/1oRHJGDS9lT1MteXfwHa3UpXhixwNOGle?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Least-to-Most Prompting

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [3]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [4]:
genai.configure(api_key=API_KEY)

In [5]:
class LeastToMostAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash")
        self.solution_chain = []

    def decompose_problem(self, problem):
        """Step 1: Break into sub-problems from easiest to hardest"""
        prompt = f"""Break down this problem into sub-problems, ordered from EASIEST to HARDEST.
        Each sub-problem should build on previous ones.

        Problem: {problem}

        List 3-6 sub-problems in order of increasing difficulty:"""

        response = self.model.generate_content(prompt).text

        # Parse sub-problems
        subproblems = []
        for line in response.split("\n"):
            line = line.strip()
            if line and (line[0].isdigit() or line.startswith("-")):
                subproblem = line.lstrip("0123456789.-) ").strip()
                if subproblem and len(subproblem) > 10:
                    subproblems.append(subproblem)

        return subproblems

    def solve_subproblem(self, subproblem, previous_solutions):
        """Step 2-3: Solve sub-problem using previous solutions"""
        if previous_solutions:
            context = "\n\n".join([
                f"Previous Solution {i+1}:\nProblem: {prev_prob}\nSolution: {prev_sol}"
                for i, (prev_prob, prev_sol) in enumerate(previous_solutions)
            ])

            prompt = f"""You have solved these simpler problems:

            {context}

            Now solve this next problem, building on what you learned:

            Problem: {subproblem}

            Solution:"""
        else:
            # First problem - no previous context
            prompt = f"""Solve this problem (the easiest/foundational one):

            Problem: {subproblem}

            Solution:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def integrate_final_solution(self, original_problem, solution_chain):
        """Step 4: Combine all solutions into final answer"""
        chain_text = "\n\n".join([
            f"Step {i+1}: {subprob}\nSolution: {solution}"
            for i, (subprob, solution) in enumerate(solution_chain)
        ])

        prompt = f"""Original Problem: {original_problem}

        Progressive Solutions (from easiest to hardest):
        {chain_text}

        Using all these progressive solutions, provide the complete final answer:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def solve(self, problem):
        """Main least-to-most prompting pipeline"""
        print(f"\n{'='*60}")
        print(f"📈 Least-to-Most Prompting")
        print(f"{'='*60}")
        print(f"Problem: {problem}\n")

        # Reset solution chain
        self.solution_chain = []

        # Step 1: Decompose from easiest to hardest
        print(f"{'─'*60}")
        print(f"STEP 1: Decomposing (Easiest → Hardest)")
        print(f"{'─'*60}\n")

        subproblems = self.decompose_problem(problem)

        print(f"Identified {len(subproblems)} sub-problems:\n")
        for i, subprob in enumerate(subproblems, 1):
            difficulty = ["🟢 Easiest", "🟡 Easy", "🟠 Medium", "🔴 Hard", "🔴 Hardest"]
            level = difficulty[min(i-1, len(difficulty)-1)]
            print(f"{i}. {level}: {subprob}")
        print()

        # Step 2-3: Solve progressively, building on previous solutions
        print(f"{'─'*60}")
        print(f"STEP 2-3: Solving Progressively")
        print(f"{'─'*60}\n")

        for i, subproblem in enumerate(subproblems, 1):
            print(f"📍 Solving sub-problem {i}/{len(subproblems)}:")
            print(f"   {subproblem[:70]}...")

            # Solve using previous solutions as context
            solution = self.solve_subproblem(subproblem, self.solution_chain)

            # Store in chain
            self.solution_chain.append((subproblem, solution))

            print(f"   ✓ Solution: {solution[:100]}...")

            if i < len(subproblems):
                print(f"   → Will use this solution for next sub-problem\n")
            else:
                print()

        # Step 4: Integrate into final solution
        print(f"{'─'*60}")
        print(f"STEP 4: Integrating Final Solution")
        print(f"{'─'*60}\n")

        final_answer = self.integrate_final_solution(problem, self.solution_chain)

        print(f"{'='*60}")
        print(f"💡 FINAL ANSWER")
        print(f"{'='*60}")
        print(final_answer)
        print()

        return final_answer

In [ ]:
# Example 1: Compositional Generalization
print("="*60)
print("EXAMPLE 1: Compositional Reasoning")
print("="*60)

agent1 = LeastToMostAgent()
agent1.solve(
    "If all roses are flowers, and some flowers fade quickly, and all things that fade quickly need water, "
    "do some roses need water?"
)


# Example 2: Math Problem (Progressive Difficulty)
print("\n" + "="*60)
print("EXAMPLE 2: Progressive Math Problem")
print("="*60)

agent2 = LeastToMostAgent()
agent2.solve(
    "Calculate the compound interest on $1000 invested at 5% annual rate for 3 years, "
    "compounded quarterly. Then calculate the total amount and the interest earned."
)


# Example 3: Code Synthesis
print("\n" + "="*60)
print("EXAMPLE 3: Progressive Code Building")
print("="*60)

agent3 = LeastToMostAgent()
agent3.solve(
    "Write a Python function that takes a list of numbers and returns the average of only "
    "the positive even numbers. Handle empty lists and edge cases."
)


# Example 4: Scientific Reasoning
print("\n" + "="*60)
print("EXAMPLE 4: Scientific Derivation")
print("="*60)

agent4 = LeastToMostAgent()
agent4.solve(
    "A car accelerates from rest at 2 m/s² for 5 seconds, then maintains constant velocity for 10 seconds, "
    "then decelerates at 1 m/s² until it stops. Calculate total distance traveled."
)


# Example 5: Educational Problem
print("\n" + "="*60)
print("EXAMPLE 5: Educational Math Tutoring")
print("="*60)

agent5 = LeastToMostAgent()
agent5.solve(
    "Solve: (3x + 2)(2x - 1) = 0. Find all values of x. Show each step clearly."
)


# Example 6: Algorithmic Thinking
print("\n" + "="*60)
print("EXAMPLE 6: Algorithm Design")
print("="*60)

agent6 = LeastToMostAgent()
agent6.solve(
    "Design an algorithm to find the longest palindromic substring in a given string. "
    "Start with understanding palindromes, then build up to the full solution."
)


# Example 7: Complex Word Problem
print("\n" + "="*60)
print("EXAMPLE 7: Multi-Step Word Problem")
print("="*60)

agent7 = LeastToMostAgent()
agent7.solve(
    "A store has a sale. Shirts are 25% off. You buy 3 shirts at $40 each. "
    "There's an additional $10 off coupon for purchases over $100. "
    "Sales tax is 8% on the final discounted price. What's the total you pay?"
)


# Example 8: Logical Puzzle
print("\n" + "="*60)
print("EXAMPLE 8: Progressive Logic Puzzle")
print("="*60)

agent8 = LeastToMostAgent()
agent8.solve(
    "In a group of 100 people, 60 like coffee, 50 like tea, and 30 like both. "
    "How many like neither? How many like only coffee? How many like at least one?"
)


print("✅ Least-to-Most Prompting Complete!")

EXAMPLE 1: Compositional Reasoning

📈 Least-to-Most Prompting
Problem: If all roses are flowers, and some flowers fade quickly, and all things that fade quickly need water, do some roses need water?

────────────────────────────────────────────────────────────
STEP 1: Decomposing (Easiest → Hardest)
────────────────────────────────────────────────────────────

Identified 5 sub-problems:

1. 🟢 Easiest: **Basic Set Relationships:** Understand what it means when "All A are B". Can you visualize or represent this relationship (e.g., with a Venn diagram)?  Focus on the idea that every element of set A is also an element of set B.
2. 🟡 Easy: **Understanding "Some":** Grasp the meaning of "Some A are B". This means at least one A is also a B, but not necessarily all of them. Visualizing this in conjunction with "All A are B" from the previous step is important.
3. 🟠 Medium: **Combining "All" and "Some":** Given "All A are B" and "Some B are C", can you determine if any A *must* be C? (The ans